In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# 示例数据
texts = [
    "The movie was fantastic and thrilling.",
    "I love the cinematography and the plot.",
    "The film was a bit too long for my taste.",
    "Amazing storyline with great acting.",
    "The plot was predictable but enjoyable."
]
labels = ["positive", "positive", "neutral", "positive", "neutral"]

# 加载预训练的句子嵌入模型
model = SentenceTransformer('all-MiniLM-L6-v2')

# 将文本转换为嵌入向量
embeddings = model.encode(texts)

# 创建Faiss索引
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# 定义检索函数
def search(query_label, k=1):
    query_embedding = model.encode([query_label])
    distances, indices = index.search(query_embedding, k)
    return [(texts[i], labels[i]) for i in indices[0]]

# 示例查询
query_label = "positive"
results = search(query_label, k=3)

print("Top matches:")
for text, label in results:
    print(f"Text: {text}, Label: {label}")

In [2]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 示例数据
texts = [
    "The movie was fantastic and thrilling.",
    "I love the cinematography and the plot.",
    "The film was a bit too long for my taste.",
    "Amazing storyline with great acting.",
    "The plot was predictable but enjoyable."
]
labels = ["positive", "positive", "neutral", "positive", "neutral"]

# 预处理文本
processed_texts = [simple_preprocess(text) for text in texts]

# 训练Word2Vec模型
w2v_model = Word2Vec(sentences=processed_texts, vector_size=100, window=5, min_count=1, workers=4)

# 将文本转换为向量
def text_to_vector(text):
    words = simple_preprocess(text)
    word_vectors = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if not word_vectors:
        return np.zeros(w2v_model.vector_size)
    return np.mean(word_vectors, axis=0)

# 计算所有文本的向量
text_vectors = np.array([text_to_vector(text) for text in texts])

# 定义检索函数
def search(query_label, k=1):
    query_vector = text_to_vector(query_label)
    similarities = cosine_similarity([query_vector], text_vectors)[0]
    top_k_indices = similarities.argsort()[-k:][::-1]
    return [(texts[i], labels[i], similarities[i]) for i in top_k_indices]

# 示例查询
query_label = "great plot and acting"
results = search(query_label, k=3)

print("Top matches:")
for text, label, similarity in results:
    print(f"Text: {text}, Label: {label}, Similarity: {similarity:.4f}")


ModuleNotFoundError: No module named 'gensim'